In [10]:
import os
#!pip install tensorflow==2.6.0
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, add, Permute, Conv2D, Add, \
    Concatenate, Multiply, LSTM, Flatten, Activation, SeparableConv2D, average
import numpy as np
# import tensorflow.keras.layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Bidirectional, Conv2D, add, TimeDistributed, GlobalAveragePooling2D, \
    Dropout, \
    Concatenate, concatenate, \
    Dense, GlobalMaxPooling2D, MaxPooling2D, Lambda, Add, Flatten, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.densenet import DenseNet201, DenseNet121, DenseNet169
import math
from tensorflow.keras.layers import Input, Bidirectional, Conv2D, add, TimeDistributed, GlobalAveragePooling2D, \
    Dropout, \
    Concatenate, concatenate, \
    Dense, GlobalMaxPooling2D, MaxPooling2D, Lambda, Add, Layer, BatchNormalization, ReLU, AveragePooling2D, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Multiply, \
    Permute, Concatenate, \
    Conv2D, Add, Activation, Lambda, add, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, add, Permute, Conv2D, Add, \
    Concatenate, Multiply, LSTM, Flatten, Activation, SeparableConv2D, average
import random
import keras_preprocessing.image


In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.config.run_functions_eagerly(True)

class ConvBlock(Layer):

    def __init__(self, filters=256, kernel_size=3, dilation_rate=1, **kwargs):
        super(ConvBlock, self).__init__(**kwargs)

        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate

        self.net = Sequential([
            Conv2D(filters, kernel_size=kernel_size, padding='same', dilation_rate=dilation_rate, use_bias=False,
                   kernel_initializer='he_normal'),
            BatchNormalization(),
            ReLU()
        ])

    def call(self, X):
        return self.net(X)

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "dilation_rate": self.dilation_rate,
        }


def AtrousSpatialPyramidPooling(X):
    B, H, W, C = X.shape

    # Image Pooling
    image_pool = AveragePooling2D(pool_size=(H, W))(X)
    image_pool = ConvBlock(kernel_size=1)(image_pool)
    image_pool = UpSampling2D(size=(H // image_pool.shape[1], W // image_pool.shape[2]),
                              )(image_pool)

    # Atrous Operaions using dilation
    conv_1 = ConvBlock(kernel_size=1, dilation_rate=1)(X)
    conv_6 = ConvBlock(kernel_size=3, dilation_rate=6)(X)
    conv_12 = ConvBlock(kernel_size=3, dilation_rate=12)(X)
    conv_18 = ConvBlock(kernel_size=3, dilation_rate=18)(X)

    # Combine All
    combined = Concatenate()([image_pool, conv_1, conv_6, conv_12, conv_18])
    processed = ConvBlock(kernel_size=1)(combined)

    # Final Output
    return processed

2024-05-19 13:34:49.291459: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79089 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:65:00.0, compute capability: 8.0


In [4]:

def cbam_block(cbam_feature, ratio=8):
    """Contains the implementation of Convolutional Block Attention Module(CBAM) block.
    As described in https://arxiv.org/abs/1807.06521.
    """

    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)

    return cbam_feature


# improved cbam block for remote sensing

def cbam_block_improved(cbam_feature, ratio=8):
    cbam_channel = channel_attention(cbam_feature, ratio)
    cbam_spatial = spatial_attention(cbam_feature)
    result1 = multiply([cbam_channel, cbam_spatial])  # was multiply previously and performance was around 92.4
    result2 = cbam_block(cbam_feature, ratio)
    # combine both types of information
    cbam_feature = Add()([result1, result2])  # previously it was  Add()
    return cbam_feature


# new attention improved
def cbam_block_improved_new(cbam_feature, ratio=8):
    cbam_channel = channel_attention(cbam_feature, ratio)
    cbam_spatial = spatial_attention(cbam_feature)
    result1 = multiply([cbam_channel, cbam_spatial])
    result2 = cbam_block(cbam_feature, ratio)
    # combine both types of information
    cbam_feature = Concatenate()([result1, result2,cbam_feature])  # previously it was  Add()
    res1 = Conv2D(filters=256, kernel_size=1, activation='relu', padding='same')(cbam_feature)
    # res1 = BatchNormalization()(res1)
    return res1


def channel_attention(input_feature, ratio=8):
    input_feature = Conv2D(256, (1, 1), activation='relu')(input_feature)
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]

    shared_layer_one = Dense(channel // ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = Reshape((1, 1, channel))(avg_pool)
    assert avg_pool.shape[1:] == (1, 1, channel)
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool.shape[1:] == (1, 1, channel // ratio)
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool.shape[1:] == (1, 1, channel)

    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1, 1, channel))(max_pool)
    assert max_pool.shape[1:] == (1, 1, channel)
    max_pool = shared_layer_one(max_pool)
    assert max_pool.shape[1:] == (1, 1, channel // ratio)
    max_pool = shared_layer_two(max_pool)
    assert max_pool.shape[1:] == (1, 1, channel)

    cbam_feature = Add()([avg_pool, max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
    # print(cbam_feature.shape)
    # print(input_feature.shape)
    # element wise application
    return multiply([input_feature, cbam_feature])


def spatial_attention(input_feature):
    input_feature = Conv2D(256, (1, 1), activation='relu')(input_feature)
    kernel_size = 7
    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2, 3, 1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters=1,
                          kernel_size=kernel_size,
                          strides=1,
                          padding='same',
                          activation='sigmoid',
                          kernel_initializer='he_normal',
                          use_bias=False)(concat)
    assert cbam_feature.shape[-1] == 1

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])


In [5]:
def load_and_crop_img(path, grayscale=False, color_mode='rgb', target_size=None,
                      interpolation='nearest'):
    """Wraps keras_preprocessing.image.utils.loag_img() and adds cropping.
    Cropping method enumarated in interpolation
    # Arguments
        path: Path to image file.
        color_mode: One of "grayscale", "rgb", "rgba". Default: "rgb".
            The desired image format.
        target_size: Either `None` (default to original size)
            or tuple of ints `(img_height, img_width)`.
        interpolation: Interpolation and crop methods used to resample and crop the image
            if the target size is different from that of the loaded image.
            Methods are delimited by ":" where first part is interpolation and second is crop
            e.g. "lanczos:random".
            Supported interpolation methods are "nearest", "bilinear", "bicubic", "lanczos",
            "box", "hamming" By default, "nearest" is used.
            Supported crop methods are "none", "center", "random".
    # Returns
        A PIL Image instance.
    # Raises
        ImportError: if PIL is not available.
        ValueError: if interpolation method is not supported.
    """

    # Decode interpolation string. Allowed Crop methods: none, center, random
    interpolation, crop = interpolation.split(":") if ":" in interpolation else (interpolation, "none")

    if crop == "none":
        return keras_preprocessing.image.utils.load_img(path,
                                                        grayscale=grayscale,
                                                        color_mode=color_mode,
                                                        target_size=target_size,
                                                        interpolation=interpolation)

    # Load original size image using Keras
    img = keras_preprocessing.image.utils.load_img(path,
                                                   grayscale=grayscale,
                                                   color_mode=color_mode,
                                                   target_size=None,
                                                   interpolation=interpolation)

    # Crop fraction of total image
    crop_fraction = 0.875
    target_width = target_size[1]
    target_height = target_size[0]

    if target_size is not None:
        if img.size != (target_width, target_height):

            if crop not in ["center", "random"]:
                raise ValueError('Invalid crop method {} specified.', crop)

            if interpolation not in keras_preprocessing.image.utils._PIL_INTERPOLATION_METHODS:
                raise ValueError(
                    'Invalid interpolation method {} specified. Supported '
                    'methods are {}'.format(interpolation,
                                            ", ".join(
                                                keras_preprocessing.image.utils._PIL_INTERPOLATION_METHODS.keys())))

            resample = keras_preprocessing.image.utils._PIL_INTERPOLATION_METHODS[interpolation]

            width, height = img.size

            # Resize keeping aspect ratio
            # result shold be no smaller than the targer size, include crop fraction overhead
            target_size_before_crop = (target_width / crop_fraction, target_height / crop_fraction)
            ratio = max(target_size_before_crop[0] / width, target_size_before_crop[1] / height)
            target_size_before_crop_keep_ratio = int(width * ratio), int(height * ratio)
            img = img.resize(target_size_before_crop_keep_ratio, resample=resample)

            width, height = img.size

            if crop == "center":
                left_corner = int(round(width / 2)) - int(round(target_width / 2))
                top_corner = int(round(height / 2)) - int(round(target_height / 2))
                return img.crop((left_corner, top_corner, left_corner + target_width, top_corner + target_height))
            elif crop == "random":
                left_shift = random.randint(0, int((width - target_width)))
                down_shift = random.randint(0, int((height - target_height)))
                return img.crop((left_shift, down_shift, target_width + left_shift, target_height + down_shift))

    return img

# Monkey patch
keras_preprocessing.image.iterator.load_img = load_and_crop_img

In [6]:
def squeeze_excite_block(input, ratio=16):
    ''' Create a channel-wise squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
    Returns: a keras tensor
    References
    -   [Squeeze and Excitation Networks](https://arxiv.org/abs/1709.01507)
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    # if K.image_data_format() == 'channels_first':
    #     se = Permute((3, 1, 2))(se)

    print(init.shape)
    print(se.shape)
    x = multiply([init, se])
    return x


def spatial_squeeze_excite_block(input):
    ''' Create a spatial squeeze-excite block
    Args:
        input: input tensor
    Returns: a keras tensor
    References
    -   [Concurrent Spatial and Channel Squeeze & Excitation in Fully Convolutional Networks](https://arxiv.org/abs/1803.02579)
    '''

    se = Conv2D(1, (1, 1), activation='sigmoid', use_bias=False,
                kernel_initializer='he_normal')(input)

    x = multiply([input, se])
    return x


# Adaptive ECA module
def eca_module(inputs, gamma=2, b=1):
    x = inputs
    # t = int(abs((K.int_shape(x)[3] * gamma) // b))
    # k = t if t % 2 else t + 1
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, x.shape[1]))(x)
    x = Conv2D(1, (3, 3), padding='same', kernel_initializer='he_normal', use_bias=False)(x)
    x = Activation('sigmoid')(x)
    x = multiply([inputs, x])
    return x


def EARCM_improved_TGRS(input_tensor):
    lev_1 = Conv2D(256, (1, 1), padding='same')(input_tensor)  # 1x1 convolution and reduce channel
    lev_2 = Conv2D(256, (2, 2), padding='same')(input_tensor)  # 1x1 convolution and reduce channel
    lev_3 = Conv2D(256, (3, 3), padding='same')(input_tensor)  # 3x3 convolution and reduce channel
    lev_4 = Conv2D(256, (4, 4), padding='same')(input_tensor)  # 4x4 convolution and reduce channel
    lev_5 = Conv2D(256, (5, 5), padding='same')(input_tensor)  # 5x5 convolution and reduce channel

    # CBAM modified block
    cbam_imp = cbam_block(input_tensor) #original cbam (cbam_block) worked to produce over 94.60% accuracy
    # proposed approach is concat
    res1 = Concatenate()(
        [lev_1, lev_3, lev_5,cbam_imp])  # prev, [lev_1, lev_3, lev_5]===94.40%

    # convolution 1 by 1 for the fusion
    conv = conv1by1(res1)

    # # eca for the selection of interesting regions after fusion
    eca = eca_module(conv)  # eca worked well! acc: 94.40%
    #eca=channel_spatial_squeeze_excite(conv) # testing this although eca worked fine to some extent
    # bn=BatchNormalization()(eca)
    return eca


def channel_spatial_squeeze_excite(input, ratio=16):
    ''' Create a spatial squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
    Returns: a keras tensor
    References
    -   [Squeeze and Excitation Networks](https://arxiv.org/abs/1709.01507)
    -   [Concurrent Spatial and Channel Squeeze & Excitation in Fully Convolutional Networks](https://arxiv.org/abs/1803.02579)
    '''

    cse = squeeze_excite_block(input, ratio)
    sse = spatial_squeeze_excite_block(input)

    x = add([cse, sse])
    return x


# Change the number of filters to 1280 for more information or keep 2688 as it is with 1 by 1 convolution,previously 1024
def conv1by1(input_tensor):
    tensor = Conv2D(256, (1, 1), activation='relu')(input_tensor)  # 256-D provides around 94% acc.
    # tensor = squeeze_excite_block(tensor)
    return tensor


def EACRM(input_tensor):
    lev_1 = Conv2D(256, (1, 1))(input_tensor)  # 1x1 convolution and reduce channel
    # print(lev_1.shape)
    lev_2 = Conv2D(256, (3, 3))(input_tensor)  # 3x3 convolution and reduce channel
    # lev_2= GlobalAveragePooling2D()(lev_2)
    # print(lev_2.shape)
    lev_3 = cbam_block_improved(input_tensor)  # CBAM modified block
    # print(lev_3.shape)
    # res1 = multiply([lev_1, lev_3])
    # proposed approach is concat
    res1 = Concatenate()(
        [lev_1, lev_3])
    # res1 = Add()(
    #     [lev_1, lev_3])
    return res1


def custom_resnet50(model, classes):
    invert11_ = model.get_layer('conv2_block3_out').output
    invert1 = model.get_layer('conv3_block4_out').output
    invert2 = model.get_layer('conv4_block6_out').output
    invert3 = model.get_layer('conv5_block3_out').output

    invert11_ = EACRM(invert11_)
    invert1 = EACRM(invert1)
    invert2 = EACRM(invert2)
    invert3 = EACRM(invert3)

    # ASPP
    invert11_ = AtrousSpatialPyramidPooling(invert11_)
    invert1 = AtrousSpatialPyramidPooling(invert1)
    invert2 = AtrousSpatialPyramidPooling(invert2)
    invert3 = AtrousSpatialPyramidPooling(invert3)

    # GAP
    invert11_ = GlobalAveragePooling2D()(invert11_)
    invert1_ = GlobalAveragePooling2D()(invert1)
    invert2_ = GlobalAveragePooling2D()(invert2)
    invert3_ = GlobalAveragePooling2D()(invert3)

    # combine all of them
    comb = concatenate([
        invert11_,
        invert1_,
        invert2_,
        invert3_
    ])
    # comb = BatchNormalization()(comb)  # added to normalize
    dense = Dense(1024, activation='relu')(comb)  # reduced the 1024->768
    dense = Dense(768, activation='relu')(dense)  # reduced the 1024->768
    # softmax
    output = Dense(classes, activation='softmax')(dense)
    model = Model(inputs=model.input, outputs=output)
    return model


def sequence_layer(input):
    print(input.shape)
    # input= conv1by1(input)
    units = input.shape[1] * input.shape[2]
    reshape = Reshape((units, input.shape[3]))(input)
    lstm_layer = LSTM(49, input_shape=(units, input.shape[3]), return_sequences=True)(reshape)
    flatten = Flatten()(lstm_layer)
    return flatten


def multi_scale_msafeb(input_tensor, ratio=4):
    print(input_tensor.shape)
    filter = input_tensor.shape[3]
    # p0=Modules.AtrousSpatialPyramidPooling(input_tensor), dilation=4, and groups=10, provided 93.80%
    pool1 = Conv2D(filters=filter / ratio, kernel_size=1, activation='relu', padding='same', dilation_rate=4,
                   groups=8)(
        input_tensor)  # previously 12 groups, could try 16 as well for more groups
    pool1_ = GlobalAveragePooling2D()(pool1)
    p1 = AtrousSpatialPyramidPooling(pool1)
    # p1 =eca_module(pool1)

    pool2 = Conv2D(filters=filter / ratio, kernel_size=3, activation='relu', padding='same', dilation_rate=4,
                   groups=8)(
        input_tensor)
    pool2_ = GlobalAveragePooling2D()(pool2)
    p2 = AtrousSpatialPyramidPooling(pool2)
    # p2=eca_module(pool2)

    pool3 = Conv2D(filters=filter / ratio, kernel_size=5, activation='relu', padding='same', dilation_rate=4,
                   groups=8)(
        input_tensor)
    # pool3 = MaxPooling2D((3, 3))(res1)
    pool3_ = GlobalAveragePooling2D()(pool3)
    p3 = AtrousSpatialPyramidPooling(pool3)
    # p3=eca_module(pool3)

    conv = concatenate([pool1, pool2, pool3, input_tensor]) #hrv1 not added before, hrv1 added in the previous version

    #conv = conv1by1(conv)

    earcm = EARCM_improved_TGRS(conv)  # our own attention block

    bn = BatchNormalization()(earcm)
    return bn, pool1_, pool2_, pool3_, p1, p2, p3


def rotation_invariant(input_tensor):
    # rotation
    o = input_tensor
    r1 = tf.image.rot90(input_tensor, k=1) # counterclockwise 90 degree
    r2 = tf.image.rot90(input_tensor, k=2) # counterclockwise 180 degree
    r3= tf.image.rot90(input_tensor,k=3) # counterclockwise 270 degree
    r4= tf.image.rot90(input_tensor,k=-1) # clockwise 90
    r5=tf.image.rot90(input_tensor,k=-2) # clockwise 180
    r6=tf.image.rot90(input_tensor, k=-3) #clockwise 270


    # msafeb
    o_m, o_p1, o_p2, o_p3, po1, po2, po3 = multi_scale_msafeb(o)
    r1_m, r1_p1, r1_p2, r1_p3, p11, p12, p13 = multi_scale_msafeb(r1)
    r2_m, r2_p1, r2_p2, r2_p3, p21, p22, p23 = multi_scale_msafeb(r2)
    r3_m, r3_p1, r3_p2, r3_p3, p31, p32, p33 = multi_scale_msafeb(r3)
    r4_m, r4_p1, r4_p2, r4_p3, p41, p42, p43 = multi_scale_msafeb(r4)
    r5_m, r5_p1, r5_p2, r5_p3, p51, p52, p53 = multi_scale_msafeb(r5)
    r6_m, r6_p1, r6_p2, r6_p3, p61, p62, p63 = multi_scale_msafeb(r6)
    
    
    # convert back to original format
    r1_m=tf.image.rot90(r1_m,k=-1)
    r2_m=tf.image.rot90(r2_m,k=-2)
    r3_m=tf.image.rot90(r3_m,k=-3)
    r4_m=tf.image.rot90(r4_m,k=1)
    r5_m=tf.image.rot90(r5_m,k=2)
    r6_m=tf.image.rot90(r6_m,k=3)
    
    return average([po1, p11, p21, p31, p41, p51, p61]),average([o_m, r1_m,r2_m,r3_m,r4_m,r5_m,r6_m]),average([o_p1, r1_p1,r2_p1,r3_p1,r4_p1,r5_p1,r6_p1]), average([o_p2, r1_p2,r2_p2,r3_p2,r4_p2,r5_p2,r6_p2]), average([o_p3, r1_p3,r2_p3,r3_p3,r4_p3,r5_p3,r6_p3])
    

def hor_ver_mul(input_tensor):
    
    # rotation
    o = input_tensor
    r1 = tf.image.rot90(input_tensor, k=1) # counterclockwise 90 degree
    r2 = tf.image.rot90(input_tensor, k=2) # counterclockwise 180 degree
    r3= tf.image.rot90(input_tensor,k=3) # counterclockwise 270 degree
    r4= tf.image.rot90(input_tensor,k=-1) # clockwise 90
    r5=tf.image.rot90(input_tensor,k=-2) # clockwise 180
    r6=tf.image.rot90(input_tensor, k=-3) #clockwise 270
    
    #horizontal and vertical multiply
    o_m=multi_scale_hrv_pooling(o)
    r1_m=multi_scale_hrv_pooling(r1)
    r2_m=multi_scale_hrv_pooling(r2)
    r3_m=multi_scale_hrv_pooling(r3)
    r4_m=multi_scale_hrv_pooling(r4)
    r5_m=multi_scale_hrv_pooling(r5)
    r6_m=multi_scale_hrv_pooling(r6)
    
    # revert back to the original format
    r1_m=tf.image.rot90(r1_m, k=-1)
    r2_m=tf.image.rot90(r2_m,k=-2)
    r3_m=tf.image.rot90(r3_m,k=-3)
    r4_m=tf.image.rot90(r4_m,k=1)
    r5_m=tf.image.rot90(r5_m,k=2)
    r6_m=tf.image.rot90(r6_m,k=3)
    
    con= average([o_m, r1_m, r2_m,r3_m,r4_m, r5_m, r6_m])
#     con=conv1by1(con)
#     con=BatchNormalization()(con)
#     return con
                           
    return concatenate([o_m, r1_m, r2_m,r3_m,r4_m, r5_m, r6_m])  #average provided over 94.50+



# horizontal pooling
def hpool(k, x, tsnr):
    hp = AveragePooling2D(pool_size=(x, k), padding='same')(tsnr)  # max pooling along ineffective
    return hp


def hpool_max(k, x, tsnr):
    hp = MaxPooling2D(pool_size=(x, k), padding='same')(tsnr)  # max pooling along ineffective
    return hp


# vertical pooling
def vpool(k, x, tsnr):
    vp = AveragePooling2D(pool_size=(x, k), padding='same')(tsnr)  # max pooling along ineffective
    return vp


def vpool_max(k, x, tsnr):
    vp = MaxPooling2D(pool_size=(x, k), padding='same')(tsnr)  # max pooling along ineffective
    return vp


# combination and use attention to capture salient information
def hvpool(hp, vp, tnsr):
    # matrix multiplication
    print(hp)
    print(vp)
    # print(tnsr)
    mul = multiply([vp, hp])
    # attention layer EARCM: TODO
    x = Activation('sigmoid')(mul)
    #x = multiply([tnsr, x])
    # mul_ = EACRM(mul)
    return x


def multi_scale_hrv_pooling(tensor):
    scales = [1]
    tensors = []
    # for i in range(0, len(scales)):
    vp = vpool(tensor.shape[1], 1, tensor)
    hp = hpool(1, tensor.shape[2], tensor)
    cmb = hvpool(hp, vp, tensor)
    # tensors.append(cmb)
    return cmb


def multi_scale_hrv_mx_pooling(tensor):
    scales = [1]
    tensors = []
    # for i in range(0, len(scales)):
    vp = vpool_max(tensor.shape[1], 1, tensor)
    hp = hpool_max(1, tensor.shape[2], tensor)
    cmb = hvpool(hp, vp, tensor)
    # tensors.append(cmb)
    return cmb


def multi_scale_hrv_mixed_pooling(tensor):
    vp = multi_scale_hrv_pooling(tensor)
    hp = multi_scale_hrv_mx_pooling(tensor)
    cmb = average([vp, hp])
    return cmb


In [7]:
# learning decay rate schedule
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.4  # in 0.5 it provided an accuracy of 80%+
    epochs_drop = 4.0  # 5.0 gives an optimal epochs_drop
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

# rotation of the input tensor
def rotation(input_tensor):
    o=input_tensor
    o_r1= tf.image.rot90(input_tensor,k=1)
    o_r2=tf.image.rot90(input_tensor, k=2)
    o_r3=tf.image.rot90(input_tensor,k=3)
    o_r11= tf.image.rot90(input_tensor,k=-1)
    o_r12=tf.image.rot90(input_tensor, k=-2)
    o_r13=tf.image.rot90(input_tensor,k=-3)
    
    
    # pass to the attention block
    a_o=EARCM_improved_TGRS(input_tensor)
    a_r1=EARCM_improved_TGRS(o_r1)
    a_r2=EARCM_improved_TGRS(o_r2)
    a_r3=EARCM_improved_TGRS(o_r3)
    a_r11=EARCM_improved_TGRS(o_r11)
    a_r12=EARCM_improved_TGRS(o_r12)
    a_r13=EARCM_improved_TGRS(o_r13)
    
    
    #rotate back
    a_r1= tf.image.rot90(a_r1,k=-1)
    a_r2=tf.image.rot90(a_r2,k=-2)
    a_r3=tf.image.rot90(a_r3,k=-3)
    a_r11= tf.image.rot90(a_r11,k=1)
    a_r12=tf.image.rot90(a_r12,k=2)
    a_r13=tf.image.rot90(a_r13,k=3)
    
    #average
    return average([a_o,a_r1,a_r2,a_r3,a_r11,a_r12,a_r13])


# single level
def multi_scale_aspp_hrv(model, classes):
    m_ = model.output
    gap=GlobalAveragePooling2D()(m_)
    seq = sequence_layer(m_)
    
    erv=rotation(m_)
    erv=GlobalAveragePooling2D()(erv)
    
    hrv=hor_ver_mul(m_)
    hrv=GlobalAveragePooling2D()(hrv)
              
    # hrv input
    #m, p1_, p2_, p3_ = multi_scale_msafeb(m_)
    aspp, m, p1_, p2_, p3_ = rotation_invariant(m_)
    m = GlobalAveragePooling2D()(m)
    aspp=GlobalAveragePooling2D()(aspp)
    #c = concatenate([m,p1_, p2_, p3_, erv])  # [m, p1_, p2_, p3_]>>94.40+ for the fold 1
    c = concatenate([m,p1_, p2_, p3_,hrv])
    # # c = BatchNormalization()(m1)
    c = Dropout(0.2)(c)
    outputs = Dense(classes, activation='softmax')(c)
    prop = Model(model.input, outputs)
    return prop



def train_ml(model, train_batches, valid_batches, classes):
    m = custom_resnet50(model, classes)
    #m = multi_scale_aspp_hrv(model, classes)
    # m = fine_tune(model, classes)
    # m=ensemble(classes)
    #m=model
    # print(m.summary())
    # m = model
    NUM_EPOCHS = 50
    LEARNING_RATE = 0.0003  # for one step
    m.compile(loss='categorical_crossentropy',  # for multiclass use categorical_crossentropy
              # optimizer=optimizers.SGD(lr=LEARNING_RATE,momentum=0.9),
              optimizer=optimizers.Adam(lr=LEARNING_RATE),
              #  optimizer=optimizers.Adam(lr_schedule),
              metrics=['acc'])

    # print(model.summary())
    # learning schedule callback
    # es = EarlyStopping(monitor='val_loss', patience=5)
    lrate = LearningRateScheduler(step_decay)
    es = EarlyStopping(monitor='val_loss', patience=5)
    mcp_save = ModelCheckpoint('/data/gpfs/projects/punim2008/data/Fold' + '_step1.h5', save_best_only=True, monitor='val_loss', mode='min')
    callbacks_list = [lrate]

    STEP_SIZE_TRAIN = train_batches.n // train_batches.batch_size
    STEP_SIZE_VALID = valid_batches.n // valid_batches.batch_size
    # lr_decay = LearningRateScheduler(schedule=lambda epoch: LEARNING_RATE * (0.9 ** epoch))
    # callbacks_list=[es]
    m.fit_generator(train_batches,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_batches,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=NUM_EPOCHS,
                    callbacks=callbacks_list
                    )

    x = m.evaluate_generator(valid_batches,
                             steps=np.ceil(len(valid_batches)),
                             use_multiprocessing=False,
                             verbose=1,
                             workers=1,
                             )
    #  print('Testing time:' + str(time.clock() - test_s_time) + 'secs.')
    m.save('model-GRSL.h5')
    return x

In [9]:
acc = []
# for i in range(0, 10):
    # print('Fold:' + str(i + 1))
    # print("*" * 100)
    # Two steps training process
    # Step 1
    # # model = DenseNet201(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    # model = m1
#     model = DenseNet201(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
model=ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
model.trainable = True
# Step 2
#     model = load_model('/data/gpfs/projects/punim2008/data/Fold_step1.h5',custom_objects={'ConvBlock':ConvBlock})
#     model.trainable = True
#     model.summary()
# exit()
# data load and train
# root_path = "D://Jagannath_dai/AID_/2_8/" + str(i + 1) + '/'
root_path = "/home/csitaula/UCM_/8_2/1/"
DATASET_PATH = root_path + 'train'
test_dir = root_path + 'val'
# DATASET_PATH = root_path/train'
# test_dir = 'root_path/val'
IMAGE_SIZE = (224, 224)
data_list = os.listdir(DATASET_PATH)
# data_list = os.listdir('D:/COVID/four_classes/splits/f4/train')
# Delete some classes that may interfere
print(len(data_list))
NUM_CLASSES = len(data_list)
BATCH_SIZE = 16  # try reducing batch size or freeze more layers if your GPU runs out of memory

# Train datagen here is a preprocessor
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   horizontal_flip=True
                                   )

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_batches = train_datagen.flow_from_directory(DATASET_PATH,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  #interpolation='lanczos:random',  # <--------- random crop
                                                  batch_size=BATCH_SIZE,
                                                  # subset="training",
                                                  seed=42,
                                                  class_mode="categorical"
                                                  # For multiclass use categorical n for binary use binary
                                                  )

valid_batches = test_datagen.flow_from_directory(test_dir,
                                                 target_size=IMAGE_SIZE,
                                                 shuffle=True,
                                                 batch_size=BATCH_SIZE,
                                                 # interpolation = 'lanczos:center', # <--------- center crop
                                                 # subset="validation",
                                                 seed=42,
                                                 class_mode="categorical"
                                                 # For multiclass use categorical n for binary use binary
                                                 )
x = train_ml(model, train_batches, valid_batches, NUM_CLASSES)
print('Test loss:', x[0])
print('Test accuracy:', x[1])
acc.append(x[1])
# del x
# del model
#exit(0)

# print the accuracy
print(acc)
a = np.array(acc)
# print('Model:' + str(j))
print('The averaged accuracy is:\n')
print(np.mean(a))
print('The std is: \n')
print(np.std(a))

21
Found 1680 images belonging to 21 classes.
Found 420 images belonging to 21 classes.


Epoch 1/50


/tmp/ipykernel_99114/801484584.py:97: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  m.fit_generator(train_batches,
/apps/easybuild-2022/easybuild/software/MPI/GCC/11.3.0/OpenMPI/4.1.4/TensorFlow/2.11.0-CUDA-11.7.0/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2024-05-19 13:36:20.862047: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2024-05-19 13:36:21.709640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


105/105 [==============================] - 88s 749ms/step - loss: 1.1701 - acc: 0.6976 - val_loss: 10.3578 - val_acc: 0.0481 - lr: 1.0000e-04
Epoch 2/50
105/105 [==============================] - 79s 756ms/step - loss: 0.1985 - acc: 0.9476 - val_loss: 10.3540 - val_acc: 0.0481 - lr: 1.0000e-04
Epoch 3/50
105/105 [==============================] - 79s 754ms/step - loss: 0.1665 - acc: 0.9399 - val_loss: 14.8075 - val_acc: 0.0505 - lr: 1.0000e-04
Epoch 4/50
105/105 [==============================] - 79s 750ms/step - loss: 0.0741 - acc: 0.9815 - val_loss: 12.4902 - val_acc: 0.0457 - lr: 4.0000e-05
Epoch 5/50
105/105 [==============================] - 79s 749ms/step - loss: 0.0340 - acc: 0.9917 - val_loss: 10.7206 - val_acc: 0.0505 - lr: 4.0000e-05
Epoch 6/50
105/105 [==============================] - 79s 754ms/step - loss: 0.0324 - acc: 0.9923 - val_loss: 6.2644 - val_acc: 0.0817 - lr: 4.0000e-05
Epoch 7/50
105/105 [==============================] - 79s 752ms/step - loss: 0.0181 - acc: 0.9

/tmp/ipykernel_99114/801484584.py:105: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  x = m.evaluate_generator(valid_batches,


27/27 [==============================] - 5s 170ms/step - loss: 0.0374 - acc: 0.9905
Test loss: 0.03740796446800232
Test accuracy: 0.9904761910438538
[0.9904761910438538]
The averaged accuracy is:

0.9904761910438538
The std is: 

0.0
